In [1]:
from pathlib import Path
from pyexplain.benchmark.file_utils import *
from pyexplain.benchmark.plot import *
from pyexplain.benchmark.check_results import check_list_folders, check_error_json_files
import IPython
import numpy as np
import matplotlib.pyplot as plt
import json
import re
pd.set_option('display.max_rows', 100)

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.weight']= 'bold'

# MAC
environment = 'MAC'
BASE_MAC_LINUX = {
    'MAC': '/Users/emiliogamba/Documents/01_VUB/01_Research/01_Shared_Projects',
    'LINUX': '/home/emilio/research/'
}

PATH_FIGURES_POST_PAPER = Path(BASE_MAC_LINUX[environment] + "01_holygrail/latex/journal/jair21/figures/")
EXPERIMENT_RESULTS = Path(BASE_MAC_LINUX[environment] + "06_HPC_Experiments/experiments/data/output/")
BASE_OUTPUT_PATH = BASE_MAC_LINUX[environment] + "/06_HPC_Experiments/experiments/data/output/"
REMOTE_EXPERIMENT_RESULTS = "/data/brussel/101/vsc10143/hpc_experiments2/experiments/data/output/"

In [2]:
all_configs_output_folders = ["PUZZLES/2022040512/"]

check_list_folders(
    output_folders=all_configs_output_folders,
    BASE_INPUT_PATH=BASE_INPUT_PATH,
    BASE_OUTPUT_PATH=BASE_OUTPUT_PATH)

path_all_configs = [BASE_OUTPUT_PATH + o for o in all_configs_output_folders]

path_config='PUZZLES/2022040512/' intput_folder_path=PosixPath('/home/emilio/research/hpc_experiments2/experiments/data/input/PUZZLES/2022040512') output_folder_path=PosixPath('/home/emilio/research/hpc_experiments2/experiments/data/output/PUZZLES/2022040512')
input_base_name='ALL_PUZZLES__job_' output_base_name='ALL_PUZZLES__results_'
checkcount(
            input_folder=/home/emilio/research//hpc_experiments2/experiments/data/input/PUZZLES/2022040512/, 
            output_folder=/home/emilio/research//hpc_experiments2/experiments/data/output/PUZZLES/2022040512/, 
            input_base_name=ALL_PUZZLES__job_, 
            output_base_name=ALL_PUZZLES__results_
        )


In [3]:
df_puzzles = folder_to_pandas_df_pickle(path_all_configs)

In [4]:
df_puzzles = corr_enhance_df(df_puzzles)

/home/emilio/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/emilio/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}

df_puzzles["explanation config"] = df_puzzles.apply(lambda row: renaming_expl_config[row["params_explanation_computer"]], axis=1)
df_puzzles["tot_lits_derived"] = df_puzzles.apply(lambda row: sum(row["lits_derived"]), axis=1)

In [10]:
df_puzzles.groupby(by=["explanation config", "params_grow", 'params_weighing', 'params_interpretation']).count()

time_totalTime  \
explanation config      params_grow            params_weighing params_interpretation                   
MUS                     ignore                 ignore          ignore                            403   
OCUS                    CORR_GREEDY            UNIFORM         ACTUAL                            403   
                        SAT                    UNIFORM         ACTUAL                            403   
OCUS+Incr. HS           CORRECTION_SUBSETS_SAT UNIFORM         ACTUAL                            403   
                        CORR_GREEDY            UNIFORM         ACTUAL                            403   
                        DISJ_MCS               UNIFORM         ACTUAL                            403   
                        MAXSAT                 UNIFORM         ACTUAL                            403   
                                                               FULL                              403   
                        SAT                    UNIFORM         ACTUAL                            403   
OUS Iter.               CORR_GREEDY            UNIFORM         ACTUAL                            403   
                        SAT                    UNIFORM         ACTUAL                            403   
OUS Iter.+Lit. Incr. HS CORRECTION_SUBSETS_SAT UNIFORM         ACTUAL                            403   
                        CORR_GREEDY            UNIFORM         ACTUAL                            403   
                        DISJ_MCS               UNIFORM         ACTUAL                            403   
                        MAXSAT                 UNIFORM         ACTUAL                            403   
                                                               FULL                              403   
                        SAT                    UNIFORM         ACTUAL                            403   
OUSb                    CORR_GREEDY            UNIFORM         ACTUAL                            403   
                        SAT                    UNIFORM         ACTUAL                            403   
OUSb+Lit. Incr. HS      CORRECTION_SUBSETS_SAT UNIFORM         ACTUAL                            403   
                        CORR_GREEDY            UNIFORM         ACTUAL                            403   
                        DISJ_MCS               UNIFORM         ACTUAL                            403   
                        MAXSAT                 UNIFORM         ACTUAL                            403   
                                                               FULL                              403   
                        SAT                    UNIFORM         ACTUAL                            403   

                                                                                      time_timeout  \
explanation config      params_grow            params_weighing params_interpretation                 
MUS                     ignore                 ignore          ignore                          403   
OCUS                    CORR_GREEDY            UNIFORM         ACTUAL                          403   
                        SAT                    UNIFORM         ACTUAL                          403   
OCUS+Incr. HS           CORRECTION_SUBSETS_SAT UNIFORM         ACTUAL                          403   
                        CORR_GREEDY            UNIFORM         ACTUAL                          403   
                        DISJ_MCS               UNIFORM         ACTUAL                          403   
                        MAXSAT                 UNIFORM         ACTUAL                          403   
                                                               FULL                            403   
                        SAT                    UNIFORM         ACTUAL                          403   
OUS Iter.               CORR_GREEDY            UNIFORM         ACTUAL                          403   
                        SAT                    UNIFOR

In [12]:
df_puzzles_timeout

,explanation config,params_grow,params_weighing,params_interpretation,time_timeout,time_totalTime
0,OUSb+Lit. Incr. HS,CORRECTION_SUBSETS_SAT,UNIFORM,ACTUAL,0,3600.00
1,OCUS,CORR_GREEDY,UNIFORM,ACTUAL,0,3600.00
2,MUS,ignore,ignore,ignore,0,0.01
3,OCUS,SAT,UNIFORM,ACTUAL,0,3600.00
4,OUS Iter.+Lit. Incr. HS,CORR_GREEDY,UNIFORM,ACTUAL,0,3600.00
...,...,...,...,...,...,...
10070,OUS Iter.,CORR_GREEDY,UNIFORM,ACTUAL,0,3600.00
10071,OUSb,CORR_GREEDY,UNIFORM,ACTUAL,0,3600.00
10072,OUSb+Lit. Incr. HS,CORR_GREEDY,UNIFORM,ACTUAL,0,3600.00
10073,OCUS+Incr. HS,CORR_GREEDY,UNIFORM,ACTUAL,0,3600.00


In [13]:
df_puzzles_timeout = df_puzzles[["explanation config", "params_grow", 'params_weighing', 'params_interpretation', 'time_timedout', 'time_totalTime']]
df_puzzles_timeout.groupby(by=["explanation config", "params_grow",'params_interpretation']).sum()

time_timedout  \
explanation config      params_grow            params_interpretation                  
MUS                     ignore                 ignore                            92   
OCUS                    CORR_GREEDY            ACTUAL                           403   
                        SAT                    ACTUAL                           403   
OCUS+Incr. HS           CORRECTION_SUBSETS_SAT ACTUAL                           403   
                        CORR_GREEDY            ACTUAL                           403   
                        DISJ_MCS               ACTUAL                           403   
                        MAXSAT                 ACTUAL                           403   
                                               FULL                             403   
                        SAT                    ACTUAL                           403   
OUS Iter.               CORR_GREEDY            ACTUAL                           403   
                        SAT                    ACTUAL                           403   
OUS Iter.+Lit. Incr. HS CORRECTION_SUBSETS_SAT ACTUAL                           403   
                        CORR_GREEDY            ACTUAL                           403   
                        DISJ_MCS               ACTUAL                           403   
                        MAXSAT                 ACTUAL                           403   
                                               FULL                             403   
                        SAT                    ACTUAL                           403   
OUSb                    CORR_GREEDY            ACTUAL                           403   
                        SAT                    ACTUAL                           403   
OUSb+Lit. Incr. HS      CORRECTION_SUBSETS_SAT ACTUAL                           403   
                        CORR_GREEDY            ACTUAL                           403   
                        DISJ_MCS               ACTUAL                           403   
                        MAXSAT                 ACTUAL                           403   
                                               FULL                             403   
                        SAT                    ACTUAL                           403   

                                                                      time_totalTime  
explanation config      params_grow            params_interpretation                  
MUS                     ignore                 ignore                      368563.02  
OCUS                    CORR_GREEDY            ACTUAL                     1450800.00  
                        SAT                    ACTUAL                     1450800.00  
OCUS+Incr. HS           CORRECTION_SUBSETS_SAT ACTUAL                     1450800.00  
                        CORR_GREEDY            ACTUAL                     1450800.00  
                        DISJ_MCS               ACTUAL                     1450800.00  
                        MAXSAT                 ACTUAL                     1450800.00  
                                               FULL                       1450800.00  
                        SAT                    ACTUAL                     1450800.00  
OUS Iter.               CORR_GREEDY            ACTUAL                     1450800.00  
                        SAT                    ACTUAL                     1450800.00  
OUS Iter.+Lit. Incr. HS CORRECTION_SUBSETS_SAT ACTUAL                     1450800.00  
                        CORR_GREEDY            ACTUAL                     1450800.00  
                        DISJ_MCS               ACTUAL                     1450800.00  
                        MAXSAT                 ACTUAL                     1450800.00  
                                               FULL                       1450800.00  
                        SAT                    ACTUAL                     1450800.00  
OUSb                    CORR_GREEDY            ACTUAL              